In [1]:
import cv2
import math
import os
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

dataset_folder = '/media/jun/data/ship'
train_folder = '/media/jun/data/ship/train_v2'
train_df = pd.read_csv(os.path.join(dataset_folder, 'train_ship_segmentations_v2.csv'))
train_df = train_df[train_df['ImageId'] != '6384c3e78.jpg']
train_df.head(5)

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [2]:
def area_isnull(x):
    if x == x:
        return 0
    else:
        return 1

train_df['isnan'] = train_df['EncodedPixels'].apply(area_isnull)
train_isship = train_df[train_df['isnan']==0]
train_nanship = train_df[train_df['isnan']==1]

In [3]:
def rle_to_mask(rle_list, SHAPE):
    tmp_flat = np.zeros(SHAPE[0]*SHAPE[1])
    if len(rle_list) == 1:
        mask = np.reshape(tmp_flat, SHAPE).T
    else:
        strt = rle_list[::2]
        length = rle_list[1::2]
        for i,v in zip(strt,length):
            tmp_flat[(int(i)-1):(int(i)-1)+int(v)] = 1.0
        mask = np.reshape(tmp_flat, SHAPE).T
    return mask

def calc_area_for_rle(rle_str):
    rle_list = [int(x) if x.isdigit() else x for x in str(rle_str).split()]
    if len(rle_list) == 1:
        return 0
    else:
        area = np.sum(rle_list[1::2])
        return area
def calc_class(area):
    area = area / (768*768)
    if area == 0:
        return 0
    elif area < 0.005:
        return 1
    elif area < 0.015:
        return 2
    elif area < 0.025:
        return 3
    elif area < 0.035:
        return 4
    elif area < 0.045:
        return 5
    else:
        return 6

In [4]:
train_isship['area'] = train_isship['EncodedPixels'].apply(calc_area_for_rle)
train_gp = train_isship.groupby('ImageId').sum()
train_gp = train_gp.reset_index()
train_gp['class'] = train_gp['area'].apply(calc_class)
train, val = train_test_split(train_gp, test_size=0.05, stratify=train_gp['class'].tolist())

val_isship = pd.DataFrame()
for i in val['ImageId']:
    val_isship = pd.concat([val_isship, train_isship[train_isship['ImageId'] == i]], axis=0)
train_isship.drop(val_isship.index)

val_isship = val_isship.groupby('ImageId')
val_nanship = train_nanship[:len(val_isship)]
train_nanship = train_nanship[len(val_isship):]

/home/jun/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [5]:
train_isship['class'] = train_isship['area'].apply(calc_class)
train_isship['class'].value_counts()

/home/jun/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


1    69770
2     8923
3     2171
4      825
5       34
Name: class, dtype: int64

In [6]:
train_5 = train_isship[train_isship['class']>=3]
train_gp5 = train_5.groupby('ImageId').sum()
train_4 = train_isship[train_isship['class']>=2]
train_gp4 = train_4.groupby('ImageId').sum()
train_3 = train_isship[(train_isship['class']==3)|(train_isship['class']==2)]
train_gp3 = train_3.groupby('ImageId').sum()
train_2 = train_isship[(train_isship['class']==2)|(train_isship['class']==1)]
train_gp2 = train_2.groupby('ImageId').sum()
train_1 = train_isship[train_isship['class']==1]
train_gp1 = train_1.groupby('ImageId').sum()

In [30]:
for image_name in train_gp5['ImageId']:
    mask_list = train_df['EncodedPixels'][train_df['ImageId'] == image_name].tolist()
    seg_mask = np.zeros((768, 768, 1))
    for item in mask_list:
        rle_list = str(item).split()
        tmp_mask = rle_to_mask(rle_list, (768, 768))
        seg_mask[:,:,0] += tmp_mask